# Housing Market Data: HUD SAFMR, FHFA HPI, Wharton WRLURI

This notebook pulls housing-market and regulatory data from three sources:

1. **HUD SAFMR** -- Small Area Fair Market Rents by ZIP code (ArcGIS FeatureServer)
2. **FHFA HPI** -- House Price Index by state and ZIP3 (bulk CSV/XLSX download)
3. **Wharton WRLURI** -- Residential Land Use Regulatory Index by community (Stata download)

All sources are **free and require no authentication**.

Results are saved to CSV and merged with the Census ACS + geospatial data from notebooks 06-07.

In [ ]:
import os
import io
import time
import zipfile
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Project root
_cwd = os.getcwd()
_project_root = _cwd if os.path.isdir(os.path.join(_cwd, "data")) or os.path.isfile(os.path.join(_cwd, ".env")) else os.path.dirname(_cwd)
DATA_DIR = os.path.join(_project_root, "data")
os.makedirs(DATA_DIR, exist_ok=True)

# FIPS / abbreviation mappings
FIPS_TO_ABBR = {
    "01": "AL", "02": "AK", "04": "AZ", "05": "AR", "06": "CA",
    "08": "CO", "09": "CT", "10": "DE", "11": "DC", "12": "FL",
    "13": "GA", "15": "HI", "16": "ID", "17": "IL", "18": "IN",
    "19": "IA", "20": "KS", "21": "KY", "22": "LA", "23": "ME",
    "24": "MD", "25": "MA", "26": "MI", "27": "MN", "28": "MS",
    "29": "MO", "30": "MT", "31": "NE", "32": "NV", "33": "NH",
    "34": "NJ", "35": "NM", "36": "NY", "37": "NC", "38": "ND",
    "39": "OH", "40": "OK", "41": "OR", "42": "PA", "44": "RI",
    "45": "SC", "46": "SD", "47": "TN", "48": "TX", "49": "UT",
    "50": "VT", "51": "VA", "53": "WA", "54": "WV", "55": "WI",
    "56": "WY", "72": "PR",
}
ABBR_TO_FIPS = {v: k for k, v in FIPS_TO_ABBR.items()}

# Full state name -> abbreviation
STATE_NAME_TO_ABBR = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "District of Columbia": "DC", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI",
    "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME",
    "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE",
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM",
    "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH",
    "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI",
    "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX",
    "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
}
ABBR_TO_STATE_NAME = {v: k for k, v in STATE_NAME_TO_ABBR.items()}

print(f"Project root: {_project_root}")
print(f"Data directory: {DATA_DIR}")

---
## 2. HUD -- Small Area Fair Market Rents (SAFMR)

The **HUD SAFMR** provides ZIP-level fair market rents by bedroom count. Data is served via
an ArcGIS FeatureServer with ~33,800 ZIP records.

- **Endpoint:** `https://services.arcgis.com/.../HUD_PDR_Small_Area_Fair_Market_Rents/FeatureServer`
- **Layer 1** = Rent data table (SAFMR_0BR through SAFMR_4BR, keyed by ZIP/ZCTA)
- **Auth:** None
- **Pagination:** 1,000 records per request; we paginate through all ~33,800 records.

In [ ]:
# ---- HUD SAFMR: Fetch all ZIP-level rents (paginated, ~34 pages of 1000) ----
HUD_SAFMR_URL = (
    "https://services.arcgis.com/VTyQ9soqVukalItT/arcgis/rest/services/"
    "HUD_PDR_Small_Area_Fair_Market_Rents/FeatureServer/1/query"
)

all_safmr_rows = []
offset = 0
page_size = 1000
page = 0

print("Fetching HUD SAFMR data (Layer 1 -- rent table)...")
print(f"Page size: {page_size}, paginating until all records fetched.\n")

while True:
    params = {
        "where": "1=1",
        "outFields": "ID,HUD_CODE,FMR_NAME,SAFMR_0BR,SAFMR_1BR,SAFMR_2BR,SAFMR_3BR,SAFMR_4BR",
        "resultOffset": offset,
        "resultRecordCount": page_size,
        "f": "json",
    }
    try:
        r = requests.get(HUD_SAFMR_URL, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        features = data.get("features", [])
        if not features:
            break
        for feat in features:
            all_safmr_rows.append(feat.get("attributes", {}))
        page += 1
        offset += page_size
        exceeded = data.get("exceededTransferLimit", False)
        print(f"  Page {page}: {len(features)} records (total so far: {len(all_safmr_rows):,})")
        if not exceeded:
            break
        time.sleep(0.3)
    except Exception as e:
        print(f"  Page {page + 1}: ERROR - {e}")
        break

df_safmr = pd.DataFrame(all_safmr_rows)

# Rename columns for clarity
df_safmr.rename(columns={
    "ID": "zcta",
    "HUD_CODE": "hud_code",
    "FMR_NAME": "fmr_name",
    "SAFMR_0BR": "safmr_0br",
    "SAFMR_1BR": "safmr_1br",
    "SAFMR_2BR": "safmr_2br",
    "SAFMR_3BR": "safmr_3br",
    "SAFMR_4BR": "safmr_4br",
}, inplace=True)

# Convert rent columns to numeric
for col in ["safmr_0br", "safmr_1br", "safmr_2br", "safmr_3br", "safmr_4br"]:
    df_safmr[col] = pd.to_numeric(df_safmr[col], errors="coerce")

# Derive state from first 2 digits of ZIP (approximate)
# Map ZIP prefix to state using first 3 digits
df_safmr["zcta"] = df_safmr["zcta"].astype(str).str.zfill(5)

print(f"\n--- HUD SAFMR: {len(df_safmr):,} ZIP/ZCTA records ---")
print(f"Columns: {list(df_safmr.columns)}")
print(f"\n2-BR Rent stats:")
print(df_safmr["safmr_2br"].describe().round(0))
df_safmr.head(10)

In [ ]:
# ---- Aggregate SAFMR to state level & save ----
# Extract state from FMR name (format: "City, ST MSA" or similar)
def extract_state_from_fmr(fmr_name):
    """Try to extract 2-letter state code from FMR area name."""
    if not isinstance(fmr_name, str):
        return None
    # Look for 2-letter state code pattern
    import re
    # Pattern: ", XX " or ", XX$" where XX is a state abbreviation
    match = re.search(r",\s*([A-Z]{2})(?:\s|$|,)", fmr_name)
    if match:
        code = match.group(1)
        if code in ABBR_TO_STATE_NAME or code == "DC":
            return code
    return None

df_safmr["state_abbr"] = df_safmr["fmr_name"].apply(extract_state_from_fmr)
matched = df_safmr["state_abbr"].notna().sum()
print(f"State extracted for {matched:,}/{len(df_safmr):,} records ({matched/len(df_safmr)*100:.1f}%)\n")

# Aggregate: median SAFMR by state
df_safmr_state = df_safmr.groupby("state_abbr").agg(
    safmr_0br_median=("safmr_0br", "median"),
    safmr_1br_median=("safmr_1br", "median"),
    safmr_2br_median=("safmr_2br", "median"),
    safmr_3br_median=("safmr_3br", "median"),
    safmr_4br_median=("safmr_4br", "median"),
    safmr_zip_count=("zcta", "count"),
).reset_index()

for col in df_safmr_state.columns:
    if "median" in col:
        df_safmr_state[col] = df_safmr_state[col].round(0).astype(int)

print(f"--- SAFMR State-Level Summary: {len(df_safmr_state)} states ---")
print(df_safmr_state.head(10).to_string(index=False))

# Save ZIP-level (full) as Parquet (large), state-level as CSV
safmr_zip_path = os.path.join(DATA_DIR, "hud_safmr_by_zip.parquet")
df_safmr.to_parquet(safmr_zip_path, index=False, engine="pyarrow")
zip_size_mb = os.path.getsize(safmr_zip_path) / 1024 / 1024
print(f"\nSaved ZIP-level: {safmr_zip_path} ({len(df_safmr):,} rows, {zip_size_mb:.1f} MB, Parquet)")

safmr_state_path = os.path.join(DATA_DIR, "hud_safmr_by_state.csv")
df_safmr_state.to_csv(safmr_state_path, index=False)
print(f"Saved state-level: {safmr_state_path} ({len(df_safmr_state)} rows, CSV)")

In [ ]:
# ---- Map 1: HUD SAFMR -- Median 2-BR Rent by State ----
fig_safmr = px.choropleth(
    df_safmr_state,
    locations="state_abbr",
    locationmode="USA-states",
    color="safmr_2br_median",
    scope="usa",
    color_continuous_scale="Reds",
    hover_name="state_abbr",
    hover_data={
        "safmr_0br_median": ":$,.0f",
        "safmr_1br_median": ":$,.0f",
        "safmr_2br_median": ":$,.0f",
        "safmr_3br_median": ":$,.0f",
        "safmr_4br_median": ":$,.0f",
        "safmr_zip_count": ":,",
        "state_abbr": False,
    },
    labels={"safmr_2br_median": "Median 2-BR SAFMR ($)"},
    title="HUD Small Area Fair Market Rent -- Median 2-BR by State",
)
fig_safmr.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
)
fig_safmr.show()

In [ ]:
# ---- Map 2: SAFMR Rent Spread (4BR - 0BR) as measure of family size premium ----
df_safmr_state["rent_spread_4br_0br"] = df_safmr_state["safmr_4br_median"] - df_safmr_state["safmr_0br_median"]

fig_spread = px.choropleth(
    df_safmr_state,
    locations="state_abbr",
    locationmode="USA-states",
    color="rent_spread_4br_0br",
    scope="usa",
    color_continuous_scale="Purples",
    hover_name="state_abbr",
    hover_data={
        "safmr_0br_median": ":$,.0f",
        "safmr_4br_median": ":$,.0f",
        "rent_spread_4br_0br": ":$,.0f",
        "state_abbr": False,
    },
    labels={"rent_spread_4br_0br": "4BR - 0BR Spread ($)"},
    title="HUD SAFMR -- Rent Spread: 4-Bedroom minus Studio by State",
)
fig_spread.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
)
fig_spread.show()

---
## 3. FHFA -- House Price Index (HPI)

The **FHFA House Price Index** tracks residential property price changes at multiple geographic levels.

- **State-level HPI:** `https://www.fhfa.gov/hpi/download/quarterly_datasets/hpi_at_state.csv`
- **ZIP3-level HPI:** `https://www.fhfa.gov/hpi/download/annual/hpi_at_zip3.xlsx`
- **Auth:** None (direct download)
- The all-transactions HPI uses 1995 Q1 = 100 as the base index.

In [ ]:
# ---- FHFA HPI: Download state-level quarterly data ----
FHFA_STATE_URL = "https://www.fhfa.gov/hpi/download/quarterly_datasets/hpi_at_state.csv"

print(f"Downloading FHFA state-level HPI from: {FHFA_STATE_URL}")
try:
    r_fhfa = requests.get(FHFA_STATE_URL, timeout=60)
    r_fhfa.raise_for_status()
    df_hpi_state = pd.read_csv(io.StringIO(r_fhfa.text))
    print(f"Downloaded: {len(df_hpi_state):,} rows x {len(df_hpi_state.columns)} columns")
    print(f"Columns: {list(df_hpi_state.columns)}")
    print(f"\nSample:")
    print(df_hpi_state.head(5).to_string(index=False))
except Exception as e:
    print(f"Download failed: {e}")
    print("Trying alternative approach...")
    # Fallback: try the master HPI file
    try:
        alt_url = "https://www.fhfa.gov/hpi/download/monthly/hpi_master.csv"
        print(f"Trying master HPI: {alt_url}")
        r_alt = requests.get(alt_url, timeout=60)
        r_alt.raise_for_status()
        df_master = pd.read_csv(io.StringIO(r_alt.text))
        # Filter to state-level, all-transactions
        df_hpi_state = df_master[
            (df_master["level"].str.contains("State", case=False, na=False)) |
            (df_master["place_name"].isin(STATE_NAME_TO_ABBR.keys()))
        ].copy()
        print(f"Filtered master to {len(df_hpi_state):,} state rows")
    except Exception as e2:
        print(f"Alternative also failed: {e2}")
        df_hpi_state = pd.DataFrame()

In [ ]:
# ---- Process FHFA state HPI data ----
if len(df_hpi_state) > 0:
    # Standardize column names (the CSV may have varying formats)
    df_hpi_state.columns = df_hpi_state.columns.str.strip().str.lower().str.replace(" ", "_")
    print(f"Columns after normalization: {list(df_hpi_state.columns)}")
    
    # Try to identify the state name and HPI value columns
    # Common column names: state/place_name, yr/year, qtr/quarter, index_nsa/hpi
    state_col = None
    for candidate in ["state", "place_name", "state_name"]:
        if candidate in df_hpi_state.columns:
            state_col = candidate
            break
    
    hpi_col = None
    for candidate in ["index_nsa", "hpi", "index", "hpi_with_appraisal", "hpi_sa"]:
        if candidate in df_hpi_state.columns:
            hpi_col = candidate
            break
    
    year_col = None
    for candidate in ["yr", "year"]:
        if candidate in df_hpi_state.columns:
            year_col = candidate
            break
    
    qtr_col = None
    for candidate in ["qtr", "quarter", "period"]:
        if candidate in df_hpi_state.columns:
            qtr_col = candidate
            break
    
    print(f"Identified columns -- state: {state_col}, hpi: {hpi_col}, year: {year_col}, quarter: {qtr_col}")
    
    if state_col and hpi_col and year_col:
        # Map state names to abbreviations
        df_hpi_state["state_abbr"] = df_hpi_state[state_col].map(STATE_NAME_TO_ABBR)
        # If that didn't work, try direct (column might already have abbreviations)
        unmapped = df_hpi_state["state_abbr"].isna().sum()
        if unmapped > len(df_hpi_state) * 0.5:
            # Might already be abbreviations
            valid_abbrs = set(ABBR_TO_STATE_NAME.keys())
            if df_hpi_state[state_col].isin(valid_abbrs).sum() > 10:
                df_hpi_state["state_abbr"] = df_hpi_state[state_col]
        
        df_hpi_state["year"] = pd.to_numeric(df_hpi_state[year_col], errors="coerce")
        df_hpi_state["hpi_value"] = pd.to_numeric(df_hpi_state[hpi_col], errors="coerce")
        if qtr_col:
            df_hpi_state["quarter"] = pd.to_numeric(df_hpi_state[qtr_col], errors="coerce")
        
        # Filter to valid state data and recent 15 years
        df_hpi_clean = df_hpi_state[
            (df_hpi_state["state_abbr"].notna()) &
            (df_hpi_state["hpi_value"].notna()) &
            (df_hpi_state["year"] >= 2010)
        ].copy()
        
        print(f"\n--- FHFA HPI State-Level (2010+): {len(df_hpi_clean):,} rows ---")
        print(f"States: {df_hpi_clean['state_abbr'].nunique()}")
        print(f"Years: {int(df_hpi_clean['year'].min())} - {int(df_hpi_clean['year'].max())}")
        print(f"HPI range: {df_hpi_clean['hpi_value'].min():.1f} - {df_hpi_clean['hpi_value'].max():.1f}")
    else:
        print("Could not identify required columns. Printing first few rows for inspection:")
        print(df_hpi_state.head())
        df_hpi_clean = pd.DataFrame()
else:
    df_hpi_clean = pd.DataFrame()
    print("No FHFA data available.")

In [ ]:
# ---- Save FHFA HPI data ----
if len(df_hpi_clean) > 0:
    # Save full quarterly data
    hpi_full_path = os.path.join(DATA_DIR, "fhfa_hpi_state_quarterly.csv")
    df_hpi_clean.to_csv(hpi_full_path, index=False)
    print(f"Saved quarterly: {hpi_full_path} ({len(df_hpi_clean):,} rows)")
    
    # Annual average HPI by state (for easier merge with census annual data)
    df_hpi_annual = df_hpi_clean.groupby(["state_abbr", "year"]).agg(
        hpi_annual_avg=("hpi_value", "mean"),
    ).reset_index()
    df_hpi_annual["hpi_annual_avg"] = df_hpi_annual["hpi_annual_avg"].round(2)
    
    hpi_annual_path = os.path.join(DATA_DIR, "fhfa_hpi_state_annual.csv")
    df_hpi_annual.to_csv(hpi_annual_path, index=False)
    print(f"Saved annual avg: {hpi_annual_path} ({len(df_hpi_annual):,} rows)")
    
    # Latest year snapshot for state-level merge
    latest_hpi_year = int(df_hpi_annual["year"].max())
    df_hpi_latest = df_hpi_annual[df_hpi_annual["year"] == latest_hpi_year].copy()
    print(f"Latest year ({latest_hpi_year}): {len(df_hpi_latest)} states")
    df_hpi_annual.tail(10)
else:
    df_hpi_annual = pd.DataFrame()
    df_hpi_latest = pd.DataFrame()
    print("No HPI data to save.")

In [ ]:
# ---- Map 3: FHFA House Price Index by State (Latest Year) ----
if len(df_hpi_latest) > 0:
    fig_hpi = px.choropleth(
        df_hpi_latest,
        locations="state_abbr",
        locationmode="USA-states",
        color="hpi_annual_avg",
        scope="usa",
        color_continuous_scale="YlOrRd",
        hover_name="state_abbr",
        hover_data={
            "hpi_annual_avg": ":.1f",
            "year": True,
            "state_abbr": False,
        },
        labels={"hpi_annual_avg": "HPI (1995Q1=100)"},
        title=f"FHFA House Price Index by State ({latest_hpi_year}, All-Transactions, 1995Q1=100)",
    )
    fig_hpi.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        paper_bgcolor="white",
        margin=dict(l=0, r=0, t=50, b=0),
        height=500,
    )
    fig_hpi.show()
else:
    print("No HPI data for map.")

In [ ]:
# ---- Map 4: FHFA HPI Animated Over Time (2010-present) ----
if len(df_hpi_annual) > 0:
    fig_hpi_anim = px.choropleth(
        df_hpi_annual.sort_values("year"),
        locations="state_abbr",
        locationmode="USA-states",
        color="hpi_annual_avg",
        scope="usa",
        animation_frame="year",
        color_continuous_scale="YlOrRd",
        range_color=[df_hpi_annual["hpi_annual_avg"].quantile(0.02),
                     df_hpi_annual["hpi_annual_avg"].quantile(0.98)],
        hover_name="state_abbr",
        hover_data={"hpi_annual_avg": ":.1f", "state_abbr": False, "year": True},
        labels={"hpi_annual_avg": "HPI (1995Q1=100)", "year": "Year"},
        title="FHFA House Price Index by State (2010-Present, Animated)",
    )
    fig_hpi_anim.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        paper_bgcolor="white",
        margin=dict(l=0, r=0, t=50, b=0),
        height=500,
    )
    fig_hpi_anim.show()
else:
    print("No HPI data for animated map.")

---
## 4. Wharton WRLURI -- Residential Land Use Regulatory Index

The **Wharton Residential Land Use Regulatory Index** measures local zoning/permit difficulty
across ~2,450 US communities.

- **Source:** Wharton faculty data page (Stata `.dta` format)
- **Download:** ZIP file containing `.dta` files; read with `pandas.read_stata()`
- **Key fields:** `WRLURI` (overall index), `LPPI` (political pressure), `LZAI` (zoning approval),
  `DRI` (density restrictions), plus state/community identifiers.
- **Auth:** None (public research data)

In [ ]:
# ---- Wharton WRLURI: Download and read Stata data file ----
WRLURI_URL = "https://real-faculty.wharton.upenn.edu/wp-content/uploads/~gyourko/WRLURI/WHARTONLANDREGULATIONDATA_1_15_2020.zip"

print(f"Downloading Wharton WRLURI data from:\n  {WRLURI_URL}")
print("(This is the 2018 survey data, ~2,450 communities)\n")

df_wrluri = pd.DataFrame()
try:
    r_wrluri = requests.get(WRLURI_URL, timeout=120)
    r_wrluri.raise_for_status()
    print(f"Downloaded: {len(r_wrluri.content) / 1024 / 1024:.1f} MB")
    
    # Extract .dta file(s) from ZIP
    with zipfile.ZipFile(io.BytesIO(r_wrluri.content)) as zf:
        dta_files = [f for f in zf.namelist() if f.endswith(".dta")]
        print(f"ZIP contents: {zf.namelist()}")
        print(f"Stata files found: {dta_files}")
        
        if dta_files:
            # Read the first .dta file
            with zf.open(dta_files[0]) as dta_f:
                df_wrluri = pd.read_stata(io.BytesIO(dta_f.read()))
            print(f"\nLoaded: {len(df_wrluri):,} rows x {len(df_wrluri.columns)} columns")
            print(f"Columns: {list(df_wrluri.columns)}")
        else:
            # Try CSV or Excel files
            other_files = [f for f in zf.namelist() if f.endswith((".csv", ".xlsx", ".xls"))]
            if other_files:
                with zf.open(other_files[0]) as of:
                    if other_files[0].endswith(".csv"):
                        df_wrluri = pd.read_csv(of)
                    else:
                        df_wrluri = pd.read_excel(io.BytesIO(of.read()))
                print(f"Loaded from {other_files[0]}: {len(df_wrluri):,} rows")

except Exception as e:
    print(f"Download/parse failed: {e}")
    print("The Wharton data server may be temporarily unavailable.")

if len(df_wrluri) > 0:
    print(f"\n--- Wharton WRLURI: {len(df_wrluri):,} communities ---")
    # Show key columns if available
    key_cols = [c for c in df_wrluri.columns if c.upper() in 
               ["WRLURI", "LPPI", "SPII", "CII", "LZAI", "LPAI", "DRI", "SRI", "EI", "AHI",
                "STATENAME", "STATE", "PLACENAME", "COMMUNITYNAME", "STAESSION", "FIPS"]]
    if key_cols:
        print(f"Key index columns found: {key_cols}")
    df_wrluri.head(10)
else:
    print("No WRLURI data loaded.")

In [ ]:
# ---- Process Wharton WRLURI -- aggregate to state level ----
if len(df_wrluri) > 0:
    # Normalize column names to lowercase
    df_wrluri.columns = df_wrluri.columns.str.strip().str.upper()
    
    # Find state column
    state_col_w = None
    for candidate in ["STATENAME", "STATE", "STAESSION"]:
        if candidate in df_wrluri.columns:
            state_col_w = candidate
            break
    
    # Find WRLURI column
    wrluri_col = None
    for candidate in ["WRLURI", "WRLURI18", "WRLURI_2018"]:
        if candidate in df_wrluri.columns:
            wrluri_col = candidate
            break
    
    print(f"State column: {state_col_w}, WRLURI column: {wrluri_col}")
    print(f"All columns: {list(df_wrluri.columns)}")
    
    if state_col_w and wrluri_col:
        # Map state names to abbreviations
        df_wrluri["state_abbr"] = df_wrluri[state_col_w].map(STATE_NAME_TO_ABBR)
        # If not matched, try treating as abbreviation directly
        if df_wrluri["state_abbr"].isna().sum() > len(df_wrluri) * 0.5:
            valid_abbrs = set(ABBR_TO_STATE_NAME.keys())
            if df_wrluri[state_col_w].isin(valid_abbrs).sum() > 10:
                df_wrluri["state_abbr"] = df_wrluri[state_col_w]
        
        df_wrluri["wrluri_value"] = pd.to_numeric(df_wrluri[wrluri_col], errors="coerce")
        
        # Find sub-index columns
        sub_indices = {}
        for idx_name in ["LPPI", "SPII", "CII", "LZAI", "LPAI", "DRI", "SRI", "EI", "AHI"]:
            for col in df_wrluri.columns:
                if idx_name in col:
                    sub_indices[idx_name.lower()] = col
                    break
        print(f"Sub-indices found: {sub_indices}")
        
        # Aggregate to state level
        agg_dict = {
            "wrluri_mean": ("wrluri_value", "mean"),
            "wrluri_median": ("wrluri_value", "median"),
            "wrluri_community_count": ("wrluri_value", "count"),
        }
        
        df_wrluri_state = df_wrluri.groupby("state_abbr").agg(**agg_dict).reset_index()
        df_wrluri_state["wrluri_mean"] = df_wrluri_state["wrluri_mean"].round(3)
        df_wrluri_state["wrluri_median"] = df_wrluri_state["wrluri_median"].round(3)
        
        print(f"\n--- WRLURI State-Level: {len(df_wrluri_state)} states ---")
        print(f"WRLURI range: {df_wrluri_state['wrluri_mean'].min():.3f} to {df_wrluri_state['wrluri_mean'].max():.3f}")
        print(f"(Positive = more restrictive, negative = less restrictive)")
        print(df_wrluri_state.sort_values("wrluri_mean", ascending=False).head(10).to_string(index=False))
    else:
        df_wrluri_state = pd.DataFrame()
        print("Could not identify state/WRLURI columns.")
else:
    df_wrluri_state = pd.DataFrame()
    print("No WRLURI data to process.")

In [ ]:
# ---- Save WRLURI data ----
if len(df_wrluri) > 0:
    wrluri_full_path = os.path.join(DATA_DIR, "wharton_wrluri_communities.csv")
    df_wrluri.to_csv(wrluri_full_path, index=False)
    print(f"Saved community-level: {wrluri_full_path} ({len(df_wrluri):,} rows)")

if len(df_wrluri_state) > 0:
    wrluri_state_path = os.path.join(DATA_DIR, "wharton_wrluri_by_state.csv")
    df_wrluri_state.to_csv(wrluri_state_path, index=False)
    print(f"Saved state-level: {wrluri_state_path} ({len(df_wrluri_state)} rows)")

if len(df_wrluri) == 0 and len(df_wrluri_state) == 0:
    print("No WRLURI data to save.")

In [ ]:
# ---- Map 5: Wharton WRLURI -- Regulatory Restrictiveness by State ----
if len(df_wrluri_state) > 0:
    fig_wrluri = px.choropleth(
        df_wrluri_state,
        locations="state_abbr",
        locationmode="USA-states",
        color="wrluri_mean",
        scope="usa",
        color_continuous_scale="RdYlGn_r",  # red = more restrictive
        hover_name="state_abbr",
        hover_data={
            "wrluri_mean": ":.3f",
            "wrluri_median": ":.3f",
            "wrluri_community_count": ":,",
            "state_abbr": False,
        },
        labels={"wrluri_mean": "WRLURI (mean)"},
        title="Wharton WRLURI -- Land Use Regulatory Restrictiveness by State (higher = more restrictive)",
    )
    fig_wrluri.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        paper_bgcolor="white",
        margin=dict(l=0, r=0, t=50, b=0),
        height=500,
    )
    fig_wrluri.show()
else:
    print("No WRLURI data for map.")

---
## 5. Combine All Data & Merge with Census + Geospatial

Join HUD SAFMR, FHFA HPI, and Wharton WRLURI at the state level, then merge with the
existing Census ACS + geospatial combined dataset from notebooks 06-07.

In [ ]:
# ---- Combine all housing/regulatory datasets at state level ----
# Start with SAFMR state-level
df_housing = df_safmr_state.copy() if len(df_safmr_state) > 0 else pd.DataFrame(columns=["state_abbr"])

# Merge FHFA HPI (latest year snapshot)
if len(df_hpi_latest) > 0:
    df_housing = df_housing.merge(
        df_hpi_latest[["state_abbr", "hpi_annual_avg"]],
        on="state_abbr", how="outer"
    )
    print(f"Added FHFA HPI: {df_hpi_latest['state_abbr'].nunique()} states")

# Merge Wharton WRLURI
if len(df_wrluri_state) > 0:
    df_housing = df_housing.merge(
        df_wrluri_state[["state_abbr", "wrluri_mean", "wrluri_median", "wrluri_community_count"]],
        on="state_abbr", how="outer"
    )
    print(f"Added Wharton WRLURI: {df_wrluri_state['state_abbr'].nunique()} states")

print(f"\n--- Combined Housing Dataset: {len(df_housing)} states x {len(df_housing.columns)} columns ---")
print(f"Columns: {list(df_housing.columns)}")
df_housing.head(10)

In [ ]:
# ---- Merge with Census + Geospatial combined data ----
census_geo_path = os.path.join(DATA_DIR, "census_geo_combined_by_state.csv")
census_state_path = os.path.join(DATA_DIR, "census_state_demographics_2010_2024.csv")

df_base = None
if os.path.exists(census_geo_path):
    df_base = pd.read_csv(census_geo_path, dtype={"state": str})
    if "state_abbr" not in df_base.columns and "state" in df_base.columns:
        df_base["state_abbr"] = df_base["state"].map(FIPS_TO_ABBR)
    print(f"Loaded census+geo: {len(df_base):,} rows x {len(df_base.columns)} cols")
elif os.path.exists(census_state_path):
    df_base = pd.read_csv(census_state_path, dtype={"state": str})
    if "state_abbr" not in df_base.columns:
        df_base["state_abbr"] = df_base["state"].map(FIPS_TO_ABBR)
    print(f"Loaded census (no geo): {len(df_base):,} rows x {len(df_base.columns)} cols")
else:
    print(f"No census data found at {census_geo_path} or {census_state_path}")
    print("Run notebooks 06-07 first. Will save housing data standalone.")

if df_base is not None:
    # Merge housing data (static per state) onto multi-year census data
    df_full = df_base.merge(df_housing, on="state_abbr", how="left")
    
    # Also merge FHFA HPI by year (time-varying)
    if len(df_hpi_annual) > 0 and "year" in df_full.columns:
        df_full["year"] = df_full["year"].astype(int)
        df_hpi_annual["year"] = df_hpi_annual["year"].astype(int)
        # Drop the static hpi_annual_avg, merge time-varying one
        if "hpi_annual_avg" in df_full.columns:
            df_full.drop(columns=["hpi_annual_avg"], inplace=True)
        df_full = df_full.merge(
            df_hpi_annual[["state_abbr", "year", "hpi_annual_avg"]],
            on=["state_abbr", "year"], how="left"
        )
        print(f"Merged time-varying HPI by state+year")
    
    print(f"\n--- Full Combined Dataset ---")
    print(f"Rows: {len(df_full):,} | Columns: {len(df_full.columns)}")
    new_cols = [c for c in df_full.columns if c not in df_base.columns]
    print(f"New columns from housing data: {new_cols}")
    df_full.head(10)
else:
    df_full = None

In [ ]:
# ---- Save all combined datasets ----
# 1) Housing-only state-level
housing_path = os.path.join(DATA_DIR, "housing_features_by_state.csv")
df_housing.to_csv(housing_path, index=False)
print(f"Saved housing-only: {housing_path} ({len(df_housing)} rows)")

# 2) Full combined (census + geo + housing)
if df_full is not None:
    full_path = os.path.join(DATA_DIR, "census_geo_housing_combined_by_state.csv")
    df_full.to_csv(full_path, index=False)
    size_mb = os.path.getsize(full_path) / 1024 / 1024
    print(f"Saved full combined: {full_path} ({len(df_full):,} rows, {size_mb:.2f} MB)")
else:
    print("Full combined not saved (no base census data).")

In [ ]:
# ---- Correlation: Housing features vs Census demographics ----
if df_full is not None and "year" in df_full.columns:
    latest = int(df_full["year"].max())
    df_latest = df_full[
        (df_full["year"] == latest) &
        (df_full["median_income"] > 0) if "median_income" in df_full.columns else True
    ].copy()
    
    corr_cols = [
        "safmr_2br_median", "hpi_annual_avg", "wrluri_mean",
        "median_income", "median_home_value", "median_rent", "population",
    ]
    available = [c for c in corr_cols if c in df_latest.columns and df_latest[c].notna().sum() > 5]
    
    if len(available) >= 2:
        corr_matrix = df_latest[available].corr().round(3)
        print(f"=== Correlation Matrix ({latest}, {len(df_latest)} states) ===")
        print(corr_matrix.to_string())
        
        print(f"\n--- Key Correlations ---")
        pairs = [
            ("safmr_2br_median", "median_rent", "SAFMR vs Census Rent"),
            ("hpi_annual_avg", "median_home_value", "HPI vs Home Value"),
            ("wrluri_mean", "median_home_value", "WRLURI vs Home Value"),
            ("wrluri_mean", "safmr_2br_median", "WRLURI vs SAFMR Rent"),
            ("hpi_annual_avg", "median_income", "HPI vs Income"),
        ]
        for c1, c2, label in pairs:
            if c1 in available and c2 in available:
                val = corr_matrix.loc[c1, c2]
                print(f"  {label:35s} {val:+.3f}")
    else:
        print(f"Not enough overlapping columns for correlation. Available: {available}")
else:
    print("Skipping correlation -- combined data not available.")

---
## 6. Summary

In [ ]:
# ---- Final Summary ----
print("=" * 80)
print("HOUSING & REGULATORY DATA COLLECTION SUMMARY")
print("=" * 80)

print(f"\n  Source              | Records      | Key Attributes")
print(f"  -------------------+-------------+-------------------------------")
print(f"  HUD SAFMR (ZIP)    | {len(df_safmr):>10,} | safmr_0br..4br by ZIP/ZCTA")
print(f"  HUD SAFMR (State)  | {len(df_safmr_state):>10,} | median rents by state")
if len(df_hpi_clean) > 0:
    print(f"  FHFA HPI (Qtrly)   | {len(df_hpi_clean):>10,} | quarterly HPI by state")
    print(f"  FHFA HPI (Annual)  | {len(df_hpi_annual):>10,} | annual avg HPI by state")
else:
    print(f"  FHFA HPI           |          0 | (download failed)")
if len(df_wrluri) > 0:
    print(f"  Wharton WRLURI     | {len(df_wrluri):>10,} | regulatory index by community")
    print(f"  WRLURI (State)     | {len(df_wrluri_state):>10,} | mean WRLURI by state")
else:
    print(f"  Wharton WRLURI     |          0 | (download failed)")
print(f"  Housing Combined   | {len(df_housing):>10,} | all housing features by state")
if df_full is not None:
    print(f"  Full Combined      | {len(df_full):>10,} | census+geo+housing ({len(df_full.columns)} columns)")

print(f"\n  Data files saved:")
saved_files = [
    "hud_safmr_by_zip.parquet",
    "hud_safmr_by_state.csv",
    "fhfa_hpi_state_quarterly.csv",
    "fhfa_hpi_state_annual.csv",
    "wharton_wrluri_communities.csv",
    "wharton_wrluri_by_state.csv",
    "housing_features_by_state.csv",
    "census_geo_housing_combined_by_state.csv",
]
for fname in saved_files:
    fpath = os.path.join(DATA_DIR, fname)
    if os.path.exists(fpath):
        size_kb = os.path.getsize(fpath) / 1024
        unit = "KB" if size_kb < 1024 else "MB"
        size_val = size_kb if size_kb < 1024 else size_kb / 1024
        print(f"    {fname:50s} {size_val:>8.1f} {unit}")
    else:
        print(f"    {fname:50s}  (not saved)")

print(f"\n  Maps generated: 5 total")
print(f"    HUD SAFMR:       Median 2-BR Rent + Rent Spread = 2 maps")
print(f"    FHFA HPI:        Latest HPI + Animated HPI = 2 maps")
print(f"    Wharton WRLURI:  Regulatory Restrictiveness = 1 map")

print("\n" + "=" * 80)
print("ALL HOUSING & REGULATORY DATA COLLECTED SUCCESSFULLY")
print("=" * 80)